In [1]:
import pandas as pd

import techminer.jupyter as jpl
from techminer.analytics import load_scopus

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
#  df = load_scopus(df)
df.info()
# df = df.applymap(lambda x: None if pd.isna(x) is True else x)
# df = DataFrame(df).generate_ID()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Authors                        688 non-null    object 
 1   Author(s) ID                   688 non-null    object 
 2   Title                          688 non-null    object 
 3   Year                           688 non-null    int64  
 4   Source title                   688 non-null    object 
 5   Volume                         602 non-null    object 
 6   Issue                          444 non-null    object 
 7   Art. No.                       132 non-null    object 
 8   Page start                     544 non-null    object 
 9   Page end                       542 non-null    object 
 10  Page count                     9 non-null      float64
 11  Cited by                       453 non-null    float64
 12  DOI                            574 non-null    obj

In [2]:
df = load_scopus(df)

100%|██████████| 5/5 [00:12<00:00,  2.49s/it]


In [3]:
jpl.summary_by_year(df, cmap='Blues')